In [2]:
import os.path as osp
from tqdm import tqdm
from collections import Counter

import pandas as pd
import numpy as np

import networkx as nx

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import ModuleList, Embedding
from torch.autograd import Variable

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data
from torch_geometric.nn import PNAConv, BatchNorm, global_add_pool
from torch_geometric.utils import negative_sampling
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
from torch_geometric.utils import degree
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import HeteroData

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

import torch
from torch_geometric.datasets import AMiner
from torch_geometric.nn import MetaPath2Vec
from torch_geometric.transforms import RandomLinkSplit

## Hetrerogenious graph construction

General data:

In [3]:
authors_edges_papers_general = pd.read_csv("../processed_data/SSORC_CS_2010_2021_authors_edges_papers_indices.csv", index_col = 0, \
                                   converters={"papers_indices": lambda x: x.strip("[]").replace("'","").split(", ")})
authors_edges_general = pd.read_csv("../processed_data/SSORC_CS_2010_2021_authors_edge_list.csv", index_col = 0)

/home/user/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
papers_features_general = pd.read_csv("../processed_data/SSORC_CS_2010_2021_papers_features_vectorized_compressed_32.csv", index_col = 0)
authors_features_general = pd.read_csv("../processed_data/SSORC_CS_2010_2021_authors_features.csv", index_col = 0)

In [5]:
aev = authors_edges_general.values
edge_to_index = {(aev[i][0], aev[i][1]):i for i in tqdm(range(len(aev)))}

100%|██████████| 30796749/30796749 [00:36<00:00, 852681.64it/s]


Local data:

In [6]:
dataset_name = 'SSORC_CS_10_21_419_2055_primus' #'SSORC_CS_10_21_3340_16830_115907_primus'

In [7]:
authors_edges_papers = pd.read_csv("../datasets/" + dataset_name + "/" + dataset_name + "_" + "authors_edges_papers_indices.csv", index_col = 0,\
                                   converters={"papers_indices": lambda x: x.strip("[]").replace("'","").split(", ")})

In [8]:
authors_graph = nx.read_edgelist("../datasets/" + dataset_name + "/" + dataset_name + "_" + "authors.edgelist", create_using = nx.DiGraph)

In [9]:
citation_graph = nx.read_edgelist("../datasets/" + dataset_name + "/" + dataset_name + "_" + "papers.edgelist", create_using = nx.DiGraph)

_________________________________________________________

In [10]:
sAe = list(authors_graph.edges)
sAe = [(int(sAe[i][0]), int(sAe[i][1])) for i in range(len(sAe))]

In [11]:
authors_edges_papers_sub_2 = [authors_edges_papers["papers_indices"][edge_to_index[sAe[i]]] for i in tqdm(range(len(sAe)))]
authors_edges_papers_sub_flat_2 = [int(item) for subarray in authors_edges_papers_sub_2 for item in subarray]
unique_papers_2 = list(set(authors_edges_papers_sub_flat_2))

100%|██████████| 1753/1753 [00:00<00:00, 27770.24it/s]


In [12]:
unique_nodes = list(authors_graph.nodes())
id_global_2_id_local_authors = {unique_nodes[i]:i for i in range(len(unique_nodes))}

In [13]:
unique_papers = list(citation_graph.nodes())
id_global_2_id_local_papers = {unique_papers[i]:i for i in range(len(unique_papers))}

In [14]:
authors_edges = list(authors_graph.edges())

author_to_paper_edges = []
for i in range(len(authors_edges_papers_sub_2)):
    node_a_1, node_a_2 = authors_edges[i]
    for j in range(len(authors_edges_papers_sub_2[i])):
        author_to_paper_edges.append([id_global_2_id_local_authors[node_a_1], \
                                      id_global_2_id_local_papers[authors_edges_papers_sub_2[i][j]]])
        author_to_paper_edges.append([id_global_2_id_local_authors[node_a_2], \
                                      id_global_2_id_local_papers[authors_edges_papers_sub_2[i][j]]])

In [15]:
authors_authors_edges = list(authors_graph.edges())

author_to_author_edges = []
for i in range(len(authors_authors_edges )):
    author_to_author_edges.append([id_global_2_id_local_authors[authors_authors_edges[i][0]],id_global_2_id_local_authors[authors_authors_edges[i][1]]])

In [16]:
citation_edges = list(citation_graph.edges())

paper_to_paper_edges = []
for i in range(len(citation_edges)):
    paper_to_paper_edges.append([id_global_2_id_local_papers[citation_edges[i][0]],id_global_2_id_local_papers[citation_edges[i][1]]])

In [17]:
papers_nodes = list(citation_graph.nodes)
papers_nodes = [int(papers_nodes[i]) for i in range(len(papers_nodes))]
papers_node_features = papers_features_general.iloc[papers_nodes, :]

In [18]:
citation_graph_features = []
for node in tqdm(citation_graph.nodes):
    citation_graph_features.append(list(papers_node_features.loc[[int(node)]].values[0]))

100%|██████████| 2055/2055 [00:00<00:00, 6626.88it/s]


In [19]:
authors_features_general = authors_features_general.set_index(["author_id"])

In [20]:
authors_nodes = list(authors_graph.nodes)
authors_nodes = [int(authors_nodes[i]) for i in range(len(authors_nodes))]
authors_node_features = authors_features_general.loc[authors_nodes]

In [21]:
authors_graph_features = []
for node in tqdm(authors_graph.nodes):
    authors_graph_features.append(list(authors_node_features.loc[[int(node)]].values[0]))

100%|██████████| 419/419 [00:00<00:00, 5663.56it/s]


### Final

In [22]:
edge_index_authors2papers = torch.Tensor(np.array(author_to_paper_edges).T).long()

In [23]:
edge_index_authors2authors = torch.Tensor(np.array(author_to_author_edges).T).long()

In [24]:
edge_index_authors2authors.shape

torch.Size([2, 1753])

In [25]:
edge_index_papers2papers = torch.Tensor(np.array(paper_to_paper_edges).T).long()

In [26]:
authors_x = torch.Tensor(np.array(authors_graph_features)).float()

In [27]:
papers_x = torch.Tensor(np.array(citation_graph_features)).float()

In [69]:
data = HeteroData()
data['author'].x = authors_x
data['paper'].x = papers_x
data['author', 'works_with', 'author'].edge_index = edge_index_authors2papers
data['author', 'of', 'paper'].edge_index = edge_index_authors2papers
data['paper', 'of', 'author'].edge_index = edge_index_authors2papers[[1, 0]]
data['paper', 'cite', 'paper'].edge_index = edge_index_papers2papers
data['paper', 'cited_by', 'paper'].edge_index = edge_index_papers2papers[[1, 0]]
print(data)

HeteroData(
  author={ x=[419, 19] },
  paper={ x=[2055, 32] },
  (author, works_with, author)={ edge_index=[2, 8892] },
  (author, of, paper)={ edge_index=[2, 8892] },
  (paper, of, author)={ edge_index=[2, 8892] },
  (paper, cite, paper)={ edge_index=[2, 5472] },
  (paper, cited_by, paper)={ edge_index=[2, 5472] }
)


In [71]:
data['author', 'works_with', 'author']

{'edge_index': tensor([[   0,    1,    2,  ...,  284,  418,  285],
        [ 851,  851, 1670,  ..., 1046, 1046, 1046]])}

In [60]:
data_true = HeteroData()
data_true['author'].x = authors_x
data_true['paper'].x = papers_x
data_true['author', 'works_with', 'author'].edge_index = edge_index_authors2authors
data_true['author', 'of', 'paper'].edge_index = edge_index_authors2papers
data_true['paper', 'of', 'author'].edge_index = edge_index_authors2papers[[1, 0]]
data_true['paper', 'cite', 'paper'].edge_index = edge_index_papers2papers
data_true['paper', 'cited_by', 'paper'].edge_index = edge_index_papers2papers[[1, 0]]
print(data_true)

HeteroData(
  author={ x=[419, 19] },
  paper={ x=[2055, 32] },
  (author, works_with, author)={ edge_index=[2, 1753] },
  (author, of, paper)={ edge_index=[2, 8892] },
  (paper, of, author)={ edge_index=[2, 8892] },
  (paper, cite, paper)={ edge_index=[2, 5472] },
  (paper, cited_by, paper)={ edge_index=[2, 5472] }
)


In [61]:
train_data, val_data, test_data = transform(data_true)

### Embeddings generation

In [29]:
transform = RandomLinkSplit(is_undirected=False, edge_types = [("paper", "cite", "paper"), ("author", "of", "paper"), ("author", "works_with", "author")],
                                                 rev_edge_types = [("paper", "cited_by", "paper"), ("paper", "of", "author"), ("author", "works_with", "author")])

In [30]:
train_data, val_data, test_data = transform(data)

In [33]:
train_data

HeteroData(
  author={ x=[419, 19] },
  paper={ x=[2055, 32] },
  (author, works_with, author)={
    edge_index=[2, 6225],
    edge_label=[12450],
    edge_label_index=[2, 12450]
  },
  (author, of, paper)={
    edge_index=[2, 6225],
    edge_label=[12450],
    edge_label_index=[2, 12450]
  },
  (paper, of, author)={ edge_index=[2, 6225] },
  (paper, cite, paper)={
    edge_index=[2, 3831],
    edge_label=[7662],
    edge_label_index=[2, 7662]
  },
  (paper, cited_by, paper)={ edge_index=[2, 3831] }
)

In [34]:
torch.save(train_data, "train_data.data")

In [35]:
torch.save(train_data, "test_data.data")

In [36]:
# metapath = [('author', 'writes', 'paper'), ('paper', 'published_in', 'venue'), ('venue', 'publishes', 'paper'), ('paper', 'written_by', 'author'),]
metapath = [('author', 'of', 'paper'), ('paper', 'cite', 'paper'), ('paper', 'cited_by', 'paper'), ('paper', 'of', 'author')]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_model = MetaPath2Vec(train_data.edge_index_dict, embedding_dim=128,
                     metapath=metapath, walk_length=40, context_size=7,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)

loader = embedding_model.loader(batch_size=128, shuffle=True, num_workers=6)
optimizer = torch.optim.SparseAdam(list(embedding_model.parameters()), lr=0.01)

In [37]:
def train(epoch, log_steps=100, eval_steps=2000):
    embedding_model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = embedding_model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print("Loss:", total_loss)
              
                  
@torch.no_grad()
def test(train_ratio=0.1):
    embedding_model.eval()

    z = embedding_model('author', batch=data['author'].y_index)
    y = data['author'].y

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return embedding_model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
                      max_iter=150)


In [38]:
for epoch in range(1, 20):
    train(epoch)
    print('Epoch:', epoch)

Loss: 91.06471157073975
Epoch: 1
Loss: 74.53839063644409
Epoch: 2
Loss: 61.147666692733765
Epoch: 3
Loss: 49.90144324302673
Epoch: 4
Loss: 41.339656352996826
Epoch: 5
Loss: 34.52973282337189
Epoch: 6
Loss: 29.487183451652527
Epoch: 7
Loss: 25.50476062297821
Epoch: 8
Loss: 22.429749011993408
Epoch: 9
Loss: 20.12497305870056
Epoch: 10
Loss: 18.31036865711212
Epoch: 11
Loss: 16.9574214220047
Epoch: 12
Loss: 15.908272564411163
Epoch: 13
Loss: 15.136503398418427
Epoch: 14
Loss: 14.559874773025513
Epoch: 15
Loss: 14.140581727027893
Epoch: 16
Loss: 13.743347525596619
Epoch: 17
Loss: 13.474844694137573
Epoch: 18
Loss: 13.250043511390686
Epoch: 19


### Link predictiobn

In [39]:
device = 'cpu'#'cuda'

In [40]:
#transform = RandomLinkSplit(is_undirected=False, edge_types = [("author", "works_with", "author")], rev_edge_types = [("author", "works_with", "author")])

In [41]:
#train_data, val_data, test_data = transform(data)

In [42]:
train_data

HeteroData(
  author={ x=[419, 19] },
  paper={ x=[2055, 32] },
  (author, works_with, author)={
    edge_index=[2, 6225],
    edge_label=[12450],
    edge_label_index=[2, 12450]
  },
  (author, of, paper)={
    edge_index=[2, 6225],
    edge_label=[12450],
    edge_label_index=[2, 12450]
  },
  (paper, of, author)={ edge_index=[2, 6225] },
  (paper, cite, paper)={
    edge_index=[2, 3831],
    edge_label=[7662],
    edge_label_index=[2, 7662]
  },
  (paper, cited_by, paper)={ edge_index=[2, 3831] }
)

In [ ]:
authors_embeddings = embedding_model("author").detach().cpu() #[0:419]
authors_embeddings.shape

In [46]:
papers_embeddings = embedding_model(node_type = "paper").detach().cpu()
papers_embeddings.shape

torch.Size([2055, 128])

In [47]:
authors_embeddings = torch.Tensor(np.array(authors_embeddings)).to(device)

In [48]:
pd.DataFrame(authors_embeddings.numpy()).to_csv("metapath_authors_embeddings_j.csv")

In [49]:
pd.DataFrame(papers_embeddings.numpy()).to_csv("metapath_papers_embeddings_j.csv")

In [50]:
embeddings = pd.read_csv("metapath_authors_embeddings_j.csv", index_col = 0)
authors_embeddings = torch.Tensor(np.array(embeddings)).to(device)

In [51]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden1 = nn.Linear(128, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.hidden3 = nn.Linear(128, 32)
        
    def forward(self):
        x = F.relu(self.hidden1(authors_embeddings))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))        
        return x

In [52]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model()
    edge_index = train_data[("author", "works_with", "author")].edge_label_index
    link_embeddings = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)  # dot product 
    link_labels = train_data[("author", "works_with", "author")].edge_label
    loss = F.binary_cross_entropy_with_logits(link_embeddings, link_labels)
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test():
    model.eval()
    perfs = []
    z = model()
    for sample in [train_data, test_data]: 
        edge_index = sample[("author", "works_with", "author")].edge_label_index
        link_embeddings = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)  # dot product 
        link_probs = link_embeddings.sigmoid()
        link_labels = sample[("author", "works_with", "author")].edge_label
        print(link_probs.cpu().round()[0:20])
        perfs.append([accuracy_score(link_labels.cpu(), link_probs.cpu().round()), f1_score(link_labels.cpu(), link_probs.cpu().round()), roc_auc_score(link_labels.cpu(), link_probs.cpu())])
    return perfs

In [ ]:
import wandb
wandb.init(project='recommender', entity='eighonet', group="metapath_baseline_tests")
wandb.run.name = "metapath2vec_baseline_400_2000_unbiased_fixed_j" + str(4)#"gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
wandb.run.save()

In [62]:
model, train_data, test_data, authors_embeddings = Net().to(device), \
                                                        train_data.to(device), test_data.to(device), \
                                                        authors_embeddings.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [63]:
epochs = 12000
for i in range(epochs):
    train_loss = train()
    if i % 30 == 0:
        metrics = test()
        print("Loss:", float(train_loss), "\nTrain:", metrics[0], "\nTest:", metrics[1])
    wandb.log({"train_acc":  metrics[0][0], "test_acc": metrics[1][0], \
                   "train_f1": metrics[0][1], "test_f1": metrics[1][1], \
                   "train_roc_auc": metrics[0][2], "test_roc_auc": metrics[1][2],})

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])
Loss: 0.6924565434455872 
Train: [0.5, 0.6666666666666666, 0.7322405277509576] 
Test: [0.5, 0.6666666666666666, 0.7227102040816328]
tensor([0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])
tensor([0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
        0., 1.])
Loss: 0.4497641921043396 
Train: [0.8057817589576547, 0.8121307601417881, 0.8942889977612494] 
Test: [0.7, 0.7229551451187334, 0.778991836734694]
tensor([0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0.])
tensor([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1.,
        0., 1.])
Loss: 0.4161703288555145 
Train: [0.8953583061889251, 0.8861320336730174, 0.9049558881261339] 
Test: [0.7142857142857143, 0.7093023255813954

KeyboardInterrupt: 

In [256]:
for j in range(2, 20):
    transform = RandomLinkSplit(is_undirected=False, edge_types = [("author", "works_with", "author")],\
                                rev_edge_types = [("author", "works_with", "author")])
    train_data, val_data, test_data = transform(data)
    
    wandb.init(project='recommender', entity='eighonet', group="metapath_baseline")
    wandb.run.name = "metapath2vec_baseline_400_2000_" + str(j)#"gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
    wandb.run.save()
    
    model, train_data, test_data, authors_embeddings_copy = Net().to(device), \
                                                        train_data.to(device), test_data.to(device), \
                                                        authors_embeddings_copy.to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
    
    epochs = 3000
    for i in range(epochs):
        train_loss = train()
        if i % 30 == 0:
            metrics = test()
            print("Loss:", float(train_loss), "\nTrain:", metrics[0], "\nTest:", metrics[1])
        wandb.log({"train_acc":  metrics[0][0], "test_acc": metrics[1][0], \
                       "train_f1": metrics[0][1], "test_f1": metrics[1][1], \
                       "train_roc_auc": metrics[0][2], "test_roc_auc": metrics[1][2],})

2021-11-29 02:27:54.983518: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])
Loss: 0.6937012672424316 
Train: [0.5, 0.6666666666666666, 0.688402024648405] 
Test: [0.5, 0.6666666666666666, 0.6803121769508844]
tensor([0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0.,
        1., 1.])
tensor([1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
        1., 1.])
Loss: 0.5126988887786865 
Train: [0.7974622550594282, 0.7766164747564217, 0.8295703880672421] 
Test: [0.7789651293588301, 0.7631103074141049, 0.8162670611055507]
tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0.,
        1., 1.])
tensor([1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1.])
Loss: 0.4749675989151001 
Train: [0.7645358175393511, 0.7645358175393511, 0.8478155861521413] 
Test: [0.7536557930258717, 0.7

test_acc,▃▁▆▇██▇▇▇▇▇▇███████████▇▇▇▅▅▄▅▅▄▂▁▂▂▂▂▂▂
test_f1,▄▅▇████▇▇██▇███████████▇▇▇▆▅▄▅▅▄▂▁▂▂▂▂▂▂
test_roc_auc,▄██████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▅▅▄▄▄▄▂▁▂▂▂▂▂▂
train_acc,▂▁▆███▇████████████████▇▇█▆▆▅▅▅▄▃▂▂▂▂▂▂▂
train_f1,▃▃▇███▇████████████████▇▇█▆▆▅▅▅▄▂▁▂▂▂▂▂▂
train_roc_auc,▃▇█████████████████████▇▇▇▆▆▄▅▄▄▂▁▂▂▂▂▂▂
test_acc,0.7604
test_f1,0.69701
test_roc_auc,0.76088
train_acc,0.80389
train_f1,0.75604


2021-11-29 02:29:05.305274: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


...

test_acc,▂▄▇██▇█▇█▇██████▇██▆▇▄▅▅▅▅▅▅▅▅▄▄▄▄▃▃▃▃▁▃
test_f1,▄▆███▇█▇█▇██████▇██▆▇▅▅▅▅▅▅▅▅▅▄▄▄▅▃▄▃▃▁▄
test_roc_auc,▄███▇▇█▇▇▆▇▇▇▇▇▇▇▇▇▆▆▃▄▄▄▄▄▄▄▄▃▃▃▄▂▃▂▂▁▃
train_acc,▂▄▇█▇████▇▇▇▇▇▇█▇▇▇▆▇▄▄▅▅▅▅▄▅▄▄▄▄▄▃▃▃▃▁▃
train_f1,▃▅▇██████▇█▇████▇██▆▇▄▅▅▅▅▅▅▅▅▄▄▄▅▃▃▃▃▁▃
train_roc_auc,▄████████▇▇▇▇▇▇▇▇▇▇▆▆▄▄▅▅▅▅▄▅▄▃▃▄▄▃▃▃▂▁▃
test_acc,0.77953
test_f1,0.72702
test_roc_auc,0.78017
train_acc,0.80437
train_f1,0.75689


2021-11-29 02:33:04.404326: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])
Loss: 0.6937079429626465 
Train: [0.5, 0.6666666666666666, 0.6950026360137157] 
Test: [0.5, 0.6666666666666666, 0.6788811128642667]
tensor([1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
        1., 1.])
tensor([1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        0., 0.])
Loss: 0.5255096554756165 
Train: [0.7679087696755541, 0.7719741202461733, 0.8473819776325285] 
Test: [0.749718785151856, 0.7531891292290628, 0.8172445120400446]
tensor([1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
        1., 1.])
tensor([1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        0., 0.])
Loss: 0.46204984188079834 
Train: [0.8103115965306779, 0.8031338556426071, 0.8665425313512269] 
Test: [0.7868391451068616, 0.

KeyboardInterrupt: 

In [40]:
%debug

> /home/user/conda/lib/python3.7/site-packages/torch_geometric/data/storage.py(51)__getattr__()
     49         except KeyError:
     50             raise AttributeError(
---> 51                 f"'{self.__class__.__name__}' object has no attribute '{key}'")
     52 
     53     def __setattr__(self, key: str, value: Any):

ipdb> edge_index
*** NameError: name 'edge_index' is not defined
ipdb> exit
